In [1]:
import xml.etree.ElementTree as Xet
import pandas as pd

cols = ["English", "Hindi"]
rows = []

xmlparse = Xet.parse('/content/NEWS2018_M-EnHi_trn.xml')
root = xmlparse.getroot()
for i in root:
  english = i.find("SourceName").text
  hindi = i.find("TargetName").text

  rows.append({"English": english,
    "Hindi": hindi
    })

df = pd.DataFrame(rows, columns=cols)

In [2]:
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [3]:
df.drop_duplicates(inplace=True)

In [4]:
df['english_sentence']=df['English'].apply(lambda x: x.lower())
df['hindi_sentence']=df['Hindi'].apply(lambda x: x.lower())

In [5]:
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub("'", '', x))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [6]:
exclude = set(string.punctuation)
df['english_sentence']=df['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [7]:
remove_digits = str.maketrans('', '', digits)
df['english_sentence']=df['english_sentence'].apply(lambda x: x.translate(remove_digits))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

df['english_sentence']=df['english_sentence'].apply(lambda x: x.strip())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.strip())
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [8]:
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [9]:
df.head()

,English,Hindi,english_sentence,hindi_sentence
0,aabhaa,आभा,aabhaa,START_ आभा _END
1,aabheer,आभीर,aabheer,START_ आभीर _END
2,aabhijaat,आभिजात,aabhijaat,START_ आभिजात _END
3,aabid,आबिद,aabid,START_ आबिद _END
4,aabshar,आबशर,aabshar,START_ आबशर _END


In [10]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in df['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in df['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [11]:
len(all_eng_words)

13406

In [12]:
len(all_hindi_words)

12971

In [13]:
df['length_eng_sentence']=df['english_sentence'].apply(lambda x:len(x.split(" ")))
df['length_hin_sentence']=df['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [14]:
df.head()

,English,Hindi,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
0,aabhaa,आभा,aabhaa,START_ आभा _END,1,3
1,aabheer,आभीर,aabheer,START_ आभीर _END,1,3
2,aabhijaat,आभिजात,aabhijaat,START_ आभिजात _END,1,3
3,aabid,आबिद,aabid,START_ आबिद _END,1,3
4,aabshar,आबशर,aabshar,START_ आबशर _END,1,3


In [15]:
df.shape

(12937, 6)

In [16]:
print("maximum length of Hindi Sentence ",max(df['length_hin_sentence']))
print("maximum length of English Sentence ",max(df['length_eng_sentence']))

maximum length of Hindi Sentence  12
maximum length of English Sentence  10


In [17]:
max_length_src=max(df['length_hin_sentence'])
max_length_tar=max(df['length_eng_sentence'])

In [18]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(13406, 12971)

In [19]:
num_decoder_tokens += 1 #for zero padding

In [20]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [21]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [22]:
df = shuffle(df)
df.head(10)

,English,Hindi,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
8545,national museum of denmark,नेशनल म्युज़ियम ऑफ डेनमार्क,national museum of denmark,START_ नेशनल म्युज़ियम ऑफ डेनमार्क _END,4,6
3513,devasree,देवश्री,devasree,START_ देवश्री _END,1,3
7551,malcolm baldrige national quality award,मैल्कम बैलरिज नेशनल क्वालिटी अवार्ड,malcolm baldrige national quality award,START_ मैल्कम बैलरिज नेशनल क्वालिटी अवार्ड _END,5,7
5157,gunung rinjani,गुनुंग रिनजनी,gunung rinjani,START_ गुनुंग रिनजनी _END,2,4
3355,daudpur,दाऊदपुर,daudpur,START_ दाऊदपुर _END,1,3
4031,ek thi ladki,एक थी लड़की,ek thi ladki,START_ एक थी लड़की _END,3,5
6112,jerrica,जेरिका,jerrica,START_ जेरिका _END,1,3
11467,sumitra,सुमित्रा,sumitra,START_ सुमित्रा _END,1,3
6690,khatta meetha,खट्टा मीठा,khatta meetha,START_ खट्टा मीठा _END,2,4
5220,gurudwara sees ganj sahib,गुरूद्वारा सीस गंज साहिब,gurudwara sees ganj sahib,START_ गुरूद्वारा सीस गंज साहिब _END,4,6


In [23]:
df.drop(['English','Hindi'],axis=1,inplace=True)

### Split the data into train and test

In [24]:
X, y = df['english_sentence'], df['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((10349,), (2588,))

### Let us save this data

In [25]:
X.to_pickle('X_train.pkl')
X.to_pickle('X_test.pkl')

In [26]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    try:
                        if t<len(target_text.split())-1:
                            decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                        if t>0:
                            # decoder target sequence (one hot encoded)
                            # does not include the START_ token
                            # Offset by one timestep
                            decoder_target_data[i, t - 1, target_token_index[word]] = 1.
                    except:
                        pass
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [27]:
latent_dim=300

In [28]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [29]:
num_decoder_tokens

12972

In [55]:
num_decoder_tokens

12972

In [57]:
decoder_inputs.shape

TensorShape([None, None])

In [30]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [32]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    4021800     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    3891600     ['input_2[0][0]']                
                                                                                              

In [48]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs =1000

In [49]:
import tensorflow as tf

In [50]:
from tensorflow.keras.callbacks import EarlyStopping

In [51]:
# early = EarlyStopping(patience=8)

In [52]:
import numpy as np

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    # callbacks=[early],
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

In [54]:
model.save_weights('/content/drive/MyDrive/Capstone_project/01_NewmyModel_weights.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [58]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [59]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        try:
          output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

          # Sample a token
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = reverse_target_char_index[sampled_token_index]
          decoded_sentence += ' '+sampled_char

          # Exit condition: either hit max length
          # or find stop character.
          if (sampled_char == '_END' or
            len(decoded_sentence) > 50):
              stop_condition = True

          # Update the target sequence (of length 1).
          target_seq = np.zeros((1,1))
          target_seq[0, 0] = sampled_token_index

          # Update states
          states_value = [h, c]
        except:
          pass

    return decoded_sentence

In [60]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [61]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: sholayar reservoir
Actual Hindi Translation:  शोलायर रिज़रवायर 
Predicted Hindi Translation:  शोलायर रिज़रवायर 


In [62]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: vrajabala
Actual Hindi Translation:  व्रजबाला 
Predicted Hindi Translation:  व्रजबाला 


In [63]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: anil
Actual Hindi Translation:  अनिल 
Predicted Hindi Translation:  अनिल 


In [45]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: tibesti
Actual Hindi Translation:  तिबेस्ती 
Predicted Hindi Translation:  तिबेस्ती 


In [46]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: vaasantee
Actual Hindi Translation:  वासंती 
Predicted Hindi Translation:  वासंती 
